# Ensemble Techniques

## Environment Setup

### Imports

In [1]:
import os
os.sys.path.append("utils")

import torch
import pandas as pd
from monai.metrics import DiceMetric
from utils.Inference import ensemble_inference

### Config

In [2]:
seed = 33
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)

## Load Test Data

In [3]:
test_pred_segs_paths = './outputs/AHNet/pred_segs/test_pred_segs'
test_gt_segs_paths = './outputs/gt_segs/test_gt_segs'
subject_ids = pd.read_csv('./data/TEST.csv')['SubjectID'].values

ah_segs, unet_segs, segresnet_segs, untr_segs, gt_segs = [], [], [], [], []
for sid in subject_ids:
    ah_channels, unet_channels, segresnet_channels, untr_channels, gt_channels = [], [], [], [], []
    for channel in ['TC', 'WT', 'ET']:
        ah_channels.append(f'./outputs/AHNet/pred_segs/test_pred_segs/pred_{sid}_{channel}.nii.gz')
        unet_channels.append(f'./outputs/UNet/pred_segs/test_pred_segs/pred_{sid}_{channel}.nii.gz')
        segresnet_channels.append(f'./outputs/SegResNet/pred_segs/test_pred_segs/pred_{sid}_{channel}.nii.gz')
        untr_channels.append(f'./outputs/UNETR/pred_segs/test_pred_segs/pred_{sid}_{channel}.nii.gz')
        gt_channels.append(f'./outputs/gt_segs/test_gt_segs/gt_{sid}_{channel}.nii.gz')
    
    ah_segs.append(ah_channels)
    unet_segs.append(unet_channels)
    segresnet_segs.append(segresnet_channels)
    untr_segs.append(untr_channels)
    gt_segs.append(gt_channels)

# Dataframe
data_df = pd.DataFrame()
data_df['SubjectID'] = subject_ids
data_df['AHNet'] = ah_segs
data_df['UNet'] = unet_segs
data_df['SegResNet'] = segresnet_segs
data_df['UNETR'] = untr_segs
data_df['GT'] = gt_segs

data_df.head()

,SubjectID,AHNet,UNet,SegResNet,UNETR,GT
0,100214B,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100214B_TC.nii.gz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100214B_WT.nii.gz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100214B_ET.nii.gz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_100214B_TC.nii.gz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100214B_WT.nii.gz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100214B_ET.nii.gz]","[./outputs/SegResNet/pred_segs/test_pred_segs/pred_100214B_TC.nii.gz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100214B_WT.nii.gz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100214B_ET.nii.gz]","[./outputs/UNETR/pred_segs/test_pred_segs/pred_100214B_TC.nii.gz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100214B_WT.nii.gz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100214B_ET.nii.gz]","[./outputs/gt_segs/test_gt_segs/gt_100214B_TC.nii.gz, ./outputs/gt_segs/test_gt_segs/gt_100214B_WT.nii.gz, ./outputs/gt_segs/test_gt_segs/gt_100214B_ET.nii.gz]"
1,100340A,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100340A_TC.nii.gz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100340A_WT.nii.gz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100340A_ET.nii.gz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_100340A_TC.nii.gz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100340A_WT.nii.gz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100340A_ET.nii.gz]","[./outputs/SegResNet/pred_segs/test_pred_segs/pred_100340A_TC.nii.gz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100340A_WT.nii.gz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100340A_ET.nii.gz]","[./outputs/UNETR/pred_segs/test_pred_segs/pred_100340A_TC.nii.gz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100340A_WT.nii.gz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100340A_ET.nii.gz]","[./outputs/gt_segs/test_gt_segs/gt_100340A_TC.nii.gz, ./outputs/gt_segs/test_gt_segs/gt_100340A_WT.nii.gz, ./outputs/gt_segs/test_gt_segs/gt_100340A_ET.nii.gz]"
2,100391A,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100391A_TC.nii.gz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100391A_WT.nii.gz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100391A_ET.nii.gz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_100391A_TC.nii.gz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100391A_WT.nii.gz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100391A_ET.nii.gz]","[./outputs/SegResNet/pred_segs/test_pred_segs/pred_100391A_TC.nii.gz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100391A_WT.nii.gz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100391A_ET.nii.gz]","[./outputs/UNETR/pred_segs/test_pred_segs/pred_100391A_TC.nii.gz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100391A_WT.nii.gz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100391A_ET.nii.gz]","[./outputs/gt_segs/test_gt_segs/gt_100391A_TC.nii.gz, ./outputs/gt_segs/test_gt_segs/gt_100391A_WT.nii.gz, ./outputs/gt_segs/test_gt_segs/gt_100391A_ET.nii.gz]"
3,100190B,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100190B_TC.nii.gz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100190B_WT.nii.gz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100190B_ET.nii.gz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_100190B_TC.nii.gz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100190B_WT.nii.gz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100190B_ET.nii.gz]","[./outputs/SegResNet/pred_segs/test_pred_segs/pred_100190B_TC.nii.gz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100190B_WT.nii.gz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100190B_ET.nii.gz]","[./outputs/UNETR/pred_segs/test_pred_segs/pred_100190B_TC.nii.gz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100190B_WT.nii.gz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100190B_ET.nii.gz]","[./outputs/gt_segs/test_gt_segs/gt_100190B_TC.nii.gz, ./outputs/gt_segs/test_gt_segs/gt_100190B_WT.nii.gz, ./outputs/gt_segs/test_gt_segs/gt_100190B_ET.nii.gz]"
4,100142A,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100142A_T

## Ensemble 1 - Average

In [4]:
def avg(predictions, _):
    stacked_predictions = torch.stack(predictions, dim=0)
    return torch.mean(stacked_predictions, dim=0)

#---------------------------------------------------------#

avg_df = ensemble_inference(data_df, avg)
avg_df.to_csv('./outputs/Ensemble/avg_test_scores.csv', index=False)
avg_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.522651,0.560597,0.572364,0.434448,3.516129,3.258065,4.193548,8.709677,8.709677,5.129032,11607.709677,11683.129032,9211.838710,17103.677419,17984.161290,13810.870968
std,0.316486,0.321923,0.330971,0.306287,3.462860,3.316301,5.237171,9.103822,9.103822,4.951376,18957.730429,19169.474793,16349.038039,25564.191374,26911.703415,22937.679130
min,0.005555,0.009937,0.006728,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.218448,0.252240,0.253042,0.130480,1.000000,1.000000,1.000000,2.000000,2.000000,1.500000,278.500000,271.000000,103.000000,802.000000,802.000000,410.000000
50%,0.640889,0.674050,0.706515,0.492227,3.000000,3.000000,4.000000,6.000000,6.000000,4.000000,1928.000000,2088.000000,1306.000000,3847.000000,4138.000000,1944.000000
75%,0.790898,0.845353,0.866648,0.678609,5.000000,4.500000,5.000000,13.500000,13.500000,7.000000,16246.000000,16055.500000,11403.000000,26305.000000,27036.500000,23520.500000
max,0.923536,0.942440,0.943803,0.884366,16.000000,16.000000,27.000000,43.000000,43.000000,21.000000,65343.000000,65754.000000,61903.000000,102770.000000,105729.000000,92221.000000


## Ensemble 2 - Weighted Average (Mean Dice Weights)

In [5]:
def weigthed_avg(predictions, label):
    # Weights 
    dice_metric = DiceMetric(include_background=True, reduction="mean")
    weights = []
    for pred in predictions:
        dice_metric(y_pred=pred, y=label)
        weights.append(dice_metric.aggregate().item())
        dice_metric.reset()
    weights = torch.tensor(weights)
    weights = weights / weights.sum()

    # Weighted average
    stacked_predictions = torch.stack(predictions, dim=0)
    t = torch.sum(stacked_predictions * weights[:, None, None, None, None, None], dim=0)
    return t

#---------------------------------------------------------#

weigthed_avg_df = ensemble_inference(data_df, weigthed_avg)
weigthed_avg_df.to_csv('./outputs/Ensemble/weighted_avg_test_scores.csv', index=False)
weigthed_avg_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.533306,0.576017,0.584808,0.434448,1.709677,1.483871,1.612903,8.709677,8.709677,5.129032,4778.387097,4786.225806,3900.354839,17103.677419,17984.161290,13810.870968
std,0.300442,0.298409,0.311887,0.306287,2.465678,2.142755,2.894563,9.103822,9.103822,4.951376,13192.788883,13253.044273,11209.350649,25564.191374,26911.703415,22937.679130
min,0.084220,0.114951,0.101266,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.218448,0.252240,0.253042,0.130480,0.000000,0.000000,0.000000,2.000000,2.000000,1.500000,0.000000,0.000000,0.000000,802.000000,802.000000,410.000000
50%,0.640889,0.674050,0.706515,0.492227,1.000000,1.000000,0.000000,6.000000,6.000000,4.000000,5.000000,2.000000,0.000000,3847.000000,4138.000000,1944.000000
75%,0.790898,0.845353,0.866648,0.678609,3.000000,3.000000,2.000000,13.500000,13.500000,7.000000,602.000000,608.500000,256.500000,26305.000000,27036.500000,23520.500000
max,0.923536,0.942440,0.943803,0.884366,10.000000,9.000000,13.000000,43.000000,43.000000,21.000000,63969.000000,64497.000000,54662.000000,102770.000000,105729.000000,92221.000000


## Ensemble 3 - Majority Voting

In [6]:
def majority_vote(predictions, _):
    stacked_tensors = torch.stack(predictions, dim=0)
    majority_votes, _ = torch.mode(stacked_tensors, dim=0)
    return majority_votes

majority_vote_df = ensemble_inference(data_df, majority_vote)
majority_vote_df.to_csv('./outputs/Ensemble/majority_vote_test_scores.csv', index=False)
majority_vote_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.580455,0.614740,0.617100,0.504830,6.161290,5.419355,7.903226,8.709677,8.709677,5.129032,14172.387097,14257.225806,11632.774194,17103.677419,17984.161290,13810.870968
std,0.292811,0.283028,0.294667,0.327607,6.361849,5.451447,8.553186,9.103822,9.103822,4.951376,21705.019766,21917.715107,19121.380415,25564.191374,26911.703415,22937.679130
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.385514,0.445795,0.438938,0.168871,2.000000,2.000000,2.000000,2.000000,2.000000,1.500000,666.500000,653.500000,413.500000,802.000000,802.000000,410.000000
50%,0.657048,0.685890,0.706022,0.599136,4.000000,4.000000,5.000000,6.000000,6.000000,4.000000,3423.000000,3427.000000,2357.000000,3847.000000,4138.000000,1944.000000
75%,0.829055,0.860853,0.871061,0.793708,7.000000,7.000000,11.000000,13.500000,13.500000,7.000000,23699.500000,23822.000000,20584.500000,26305.000000,27036.500000,23520.500000
max,0.950684,0.955906,0.958353,0.937794,24.000000,22.000000,32.000000,43.000000,43.000000,21.000000,80558.000000,80731.000000,71546.000000,102770.000000,105729.000000,92221.000000
